In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.linear_model import Ridge, ElasticNet, SGDRegressor, LinearRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RANSACRegressor, TheilSenRegressor

# Load the data
file_path = "../Data_ML/4_out_csvs_regression/output_bfill_imputed.csv"
data = pd.read_csv(file_path)

# Define columns
data_columns = [
    'OF2', 'OF3', 'OF4', 'OF5', 'OF6', 'OF7', 'OF8', 'OF9', 'OF10', 'OF11', 'OF13', 'OF14', 'OF15', 'OF16', 'OF17',
    'OF18', 'OF19', 'OF20', 'OF21', 'OF22', 'OF23', 'OF24', 'OF25', 'OF26', 'OF27', 'OF28', 'OF30', 'OF31',
    'OF33', 'OF34', 'OF37', 'OF38', 'F1', 'F2', 'F3_a', 'F3_b', 'F3_c', 'F3_d', 'F3_e', 'F3_f', 'F3_g', 'F4', 'F5', 'F6',
    'F7', 'F8', 'F9', 'F10', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23',
    'F24', 'F25', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39', 'F40',
    'F41', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F56', 'F57',
    'F58', 'F59', 'F62', 'F63', 'F64', 'F65', 'F67', 'F68', 'S1', 'S2', 'S4', 'S5','Provincial_Class','Federal_Class','Regime','Vegetation_Type','Vegetation_Cover','Woody_Canopy_Cover','Moss_Cover','Phragmites','Soil_Type','Surface_Water_Present','Saturation_Depth','Living_Moss_Depth','Organic_Depth','Hydrogeomorphic_Class'
]
#results_columns = ['PR_Benefit', 'NR_Benefit', 'SR_Benefit', 'WS_Benefit', 'SFST_Benefit', 'PR_Benefit', 'NR_Benefit', 'SR', 'WS_Benefit', 'SFST_Benefit']
results_columns = ['PR','NR','WS','SFST']


for result in results_columns:
    print(result)
    #print(result)
    X = data[data_columns]
    y = data[result]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define models
    models = [
        Ridge(), DecisionTreeRegressor(), GradientBoostingRegressor(), RandomForestRegressor(), AdaBoostRegressor(), 
        KNeighborsRegressor(), MLPRegressor(max_iter=200), ElasticNet(max_iter=1000), 
        SVR(cache_size=1000), BayesianRidge(max_iter=1000), KernelRidge(), LinearRegression(), RANSACRegressor(), 
        TheilSenRegressor()
    ]

    # Create dictionaries to store feature importances for all models
    all_feature_importances_avg = {col: [] for col in X.columns}
    points_per_feature = {col: 0 for col in X.columns}
    top_bottom_features_per_model = {type(model).__name__: {'Top 2': [], 'Bottom 2': []} for model in models}

    # Iterate over models
    for model in models:
        #print(str(model))
        # Train the model
        model.fit(X_train, y_train)

        # Perform permutation feature importance analysis
        perm_importance = permutation_importance(model, X_test, y_test, n_repeats=30, random_state=42)

        # Get feature importances
        feature_importances = perm_importance.importances_mean

        # Get indices of features sorted by importance
        most_important_indices = feature_importances.argsort()[-2:][::-1]
        least_important_indices = feature_importances.argsort()[:2]
        
        important_indices = feature_importances.argsort()[:]

        # Calculate points for each feature
        for i in range(len(important_indices)):
            points_per_feature[X.columns[important_indices[i]]] += i

        # Store top 2 and bottom 2 important features
        for idx in most_important_indices:
            top_bottom_features_per_model[type(model).__name__]['Top 2'].append((X.columns[idx], feature_importances[idx]))
        for idx in least_important_indices:
            top_bottom_features_per_model[type(model).__name__]['Bottom 2'].append((X.columns[idx], feature_importances[idx]))

        # Append the feature importances to the dictionary
        for idx, col in enumerate(X.columns):
            all_feature_importances_avg[col].append(feature_importances[idx])

    # Calculate average feature importance across all models
    for col, importances in all_feature_importances_avg.items():
        if len(importances) > 0:
            all_feature_importances_avg[col] = np.mean(importances)

    # Get indices of features sorted by average importance
    sorted_indices_avg = np.argsort(list(all_feature_importances_avg.values()))[::-1]

    # Sort features based on points
    sorted_features = sorted(points_per_feature.keys(), key=lambda x: points_per_feature[x], reverse=True)
    sorted_average_importances = [all_feature_importances_avg[feature] for feature in sorted_features]
    sorted_points = [points_per_feature[feature] for feature in sorted_features]

    # Plot the results
    fig, ax1 = plt.subplots(figsize=(20, 8))

    color = 'tab:blue'
    ax1.set_xlabel('Features')
    plt.xticks(rotation=90, ha='center')
    ax1.set_ylabel('Average Importance (Log Scale)', color=color)
    ax1.set_yscale('log')
    ax1.bar(sorted_features, sorted_average_importances, color=color, alpha=0.6)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:red'
    ax2.set_ylabel('Points', color=color)  # we already handled the x-label with ax1
    ax2.plot(sorted_features, sorted_points, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.xticks(rotation=90, ha='center')
    plt.title(f'Feature Importance and Points per Feature for {result}')
    plt.savefig(f'feature_importance_points_plot_{result}.png')

    # Generate LaTeX table
    latex_code = r"""


\begin{table}[h!]
\centering
\begin{tabular}{|l|l|l|}
\hline
\textbf{Model} & \textbf{Top 2 Important Features} & \textbf{Bottom 2 Important Features} \\
\hline
"""

    for model, features in top_bottom_features_per_model.items():
        latex_code += f"{model} & \n"
        latex_code += "\\begin{tabular}[c]{@{}l@{}}\n"
        for feature, importance in features['Top 2']:
            latex_code += f"'{feature}': {importance:.6f}\\\\\n"
        latex_code += "\\end{tabular} & \n"
        latex_code += "\\begin{tabular}[c]{@{}l@{}}\n"
        for feature, importance in features['Bottom 2']:
            latex_code += f"'{feature}': {importance:.6f}\\\\\n"
        latex_code += "\\end{tabular} \\\\\n"
        latex_code += "\\hline\n"

    # Average importance
    latex_code += r"\textbf{Average Importance} & \begin{tabular}[c]{@{}l@{}}"
    for idx in sorted_indices_avg[:4]:
        latex_code += f"'Feature {list(all_feature_importances_avg.keys())[idx]}': {list(all_feature_importances_avg.values())[idx]:.6f}\\\\\n"
    latex_code += r"\end{tabular} & \begin{tabular}[c]{@{}l@{}}"
    for idx in sorted_indices_avg[-4:]:
        latex_code += f"'Feature {list(all_feature_importances_avg.keys())[idx]}': {list(all_feature_importances_avg.values())[idx]:.6f}\\\\\n"
    latex_code += r"\end{tabular} \\ \hline"

    # Voting system
    sorted_points_features = sorted(points_per_feature.keys(), key=lambda x: points_per_feature[x], reverse=True)
    latex_code += r"\textbf{Voting System} & \begin{tabular}[c]{@{}l@{}}"
    for feature in sorted_points_features[:4]:
        latex_code += f"'Feature {feature}': {points_per_feature[feature]} points\\\\\n"
    latex_code += r"\end{tabular} & \begin{tabular}[c]{@{}l@{}}"
    for feature in sorted_points_features[-4:]:
        latex_code += f"'Feature {feature}': {points_per_feature[feature]} points\\\\\n"
    latex_code += r"\end{tabular} \\ \hline"

    latex_code += r"""
\end{tabular}
\caption{Features for """ + result + r""" Models}
\label{tab:important_features_""" + result + r"""}
\end{table}

"""

    # Save LaTeX code to file
    with open(f'feature_importance_table_{result}.tex', 'w') as f:
        f.write(latex_code)
    
    # Print LaTeX code
    print(latex_code)


PR


C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
